In [1]:
import PyPDF2
import re
import nltk
import string
from nltk.corpus import stopwords
from collections import Counter

# turn pdf into text

In [2]:
def pdf_to_text(file):
    pdf_file = open(file, 'rb')
    read_pdf = PyPDF2.PdfFileReader(pdf_file)
    number_of_pages = read_pdf.getNumPages()
    page_content = []
    for i in range(0, number_of_pages):
        page = read_pdf.getPage(i)
        page = page.extractText()
        page = page.lower()
        page_content.append(page)
    page_content = ''.join(page_content)
    return page_content

# attempt to brute-force locate notification format headers. use the headers to find the relevant paragraph with information on the source of the breach and the data type breached

In [3]:
def try_notice_headers_source(text):
    text_format = text
    text = text.replace('\n','')
    notice_headers = ['what happened','what we are doing', 'what information was involved', 'what you can do', 'for more information']
    working = [True for x in notice_headers if x in text]
    if working:
        k = text.index(notice_headers[0])
        try:
            l = text.index(notice_headers[1])
        except:
            l = text.index(notice_headers[2])
        source = text[k:l]        
        return source
    else: 
        return False

In [4]:
def try_notice_headers_datatype(text):
    text_format = text
    text = text.replace('\n','')
    notice_headers = ['what happened','what we are doing', 'what information was involved', 'what you can do', 'for more information']
    working = [True for x in notice_headers if x in text]
    if working:
        k = text.index(notice_headers[3])
        try:
            l = text.index(notice_headers[4])
        except:
            l = text.index(notice_headers[5])
        source = text[k:l]        
        return source
    else: 
        return False

# clean the output dictionary
From the above functions I created two dictionaries, each mapping the document number (key) to the document string under the appropriate header (value). I then cleaned these dictionaries to produce wordcounts of the overall paragraphs to use for further analysis

In [5]:
def clean_dict(dct):
    dct_refined = dct.copy()
    for key in dct:
        if type(dct[key]) == bool:
            del dct_refined[key]
    for key in dct:
        try:
            if len(key)<20:
                del dct_refined[key]
        except:
            continue
    return dct_refined

create a counter to count number of words

In [6]:
def make_counter(source_dct):
    l = []
    for key in source_dct:
        st = source_dct[key]
        words = nltk.word_tokenize(st)
        l.extend(words)
    word_counts = Counter(l)
    for item in list(string.punctuation):
            if item in word_counts:
                del word_counts[item]
    for item in stop_words:
            if item in word_counts:
                del word_counts[item]
    return word_counts